## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, Model

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    # Add a channels dimension
    x = x[..., tf.newaxis].astype("float32")
    x_test = x_test[..., tf.newaxis].astype("float32")
    # 使用 tf.data 来将数据集切分为 batch 以及混淆数据集：
    train_ds = tf.data.Dataset.from_tensor_slices(
    (x, y)).shuffle(10000).batch(32)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1000)
    return train_ds, test_ds

2024-10-31 06:54:50.186257: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## 建立模型

In [2]:
class myModel(Model):
    def __init__(self):
        super(myModel, self).__init__()
        #################### 填空一
        '''声明模型对应的参数'''
        #卷积层1： patch 7x7, in size 1, out size 32，  滑动步长 1， padding  same, 激活函数 relu (tf.keras.layers.Conv2D)
        self.conv1 = layers.Conv2D(filters=32, kernel_size=(7, 7), strides=1, padding='same', activation='relu')
        #池化层1：  滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式  same (layers.MaxPool2D)
        self.pool1 = layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        #卷积层2： patch 5x5, in size 32, out size 64，  滑动步长 1， padding 方式 same, 激活函数 relu
        self.conv2 = layers.Conv2D(filters=64, kernel_size=(5, 5), strides=1, padding='same', activation='relu')
        #池化层2：  滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式  same
        self.pool2 = layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same')
        # 铺平特征映射
        self.flatten = layers.Flatten()
        # 全连接层 1: output dim 1024, 激活函数 relu： (tf.keras.layers.Dense)
        self.d1 = layers.Dense(1024, activation='relu')
        # 全连接层 2: output dim 10：
        self.d2 = layers.Dense(10)

        ####################
    def call(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x
        ####################
        return logits
        
model = myModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

2024-10-31 06:54:51.452972: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-10-31 06:54:51.463431: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:b2:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2024-10-31 06:54:51.463462: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-10-31 06:54:51.470162: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-10-31 06:54:51.470211: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-10-31 06:54:51.472716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuff

## 计算 loss

In [3]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_object(y, predictions)

    # compute gradient
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y, predictions)

@tf.function
def test_step(model,images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## 实际训练

In [4]:
train_ds, test_ds = mnist_dataset()
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_one_step(model,optimizer,images, labels)

  for test_images, test_labels in test_ds:
    test_step(model,test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}',
  )

2024-10-31 06:54:53.845013: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-10-31 06:54:53.850348: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2400000000 Hz
2024-10-31 06:54:53.914510: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2024-10-31 06:54:54.622589: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2024-10-31 06:54:55.787269: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-10-31 06:54:56.733140: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2024-10-31 06:54:56.791505: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1, Loss: 0.09890367835760117, Accuracy: 96.95833587646484, Test Loss: 0.03737422823905945, Test Accuracy: 98.73999786376953
Epoch 2, Loss: 0.03754333406686783, Accuracy: 98.83333587646484, Test Loss: 0.03470778092741966, Test Accuracy: 98.93000030517578
Epoch 3, Loss: 0.023771073669195175, Accuracy: 99.24832916259766, Test Loss: 0.028785109519958496, Test Accuracy: 99.08999633789062
Epoch 4, Loss: 0.019406188279390335, Accuracy: 99.37000274658203, Test Loss: 0.029942985624074936, Test Accuracy: 99.06999969482422
Epoch 5, Loss: 0.014598574489355087, Accuracy: 99.55500030517578, Test Loss: 0.039276741445064545, Test Accuracy: 99.04999542236328
